In [2]:
import pandas as pd

In [67]:
cases = pd.read_csv('dwh_csv/f_cases.csv')
general = pd.read_csv('dwh_csv/f_general_metrics.csv')
inmates = pd.read_csv('dwh_csv/d_inmate.csv')
dates = pd.read_csv('dwh_csv/d_date.csv')
old_cases = pd.read_csv('current_data/Cases.csv')

In [69]:
# Check the common columns between the two DataFrames
common_columns = set(cases.columns).intersection(set(old_cases.columns))
print(f"Common columns: {common_columns}")


Common columns: {'juvenile_misdemeanor_count', 'juvenile_other_offense_count', ' Case_id', 'days_from_compas_to_current_offense', 'prior_offense_count', 'days_between_screening_and_arrest', 'juvenile_felony_count', 'is_violent_recidivist', 'is_recidivist'}


In [80]:
print(cases.columns)
print(old_cases.columns)


Index([' Case_id', 'juvenile_felony_count', 'juvenile_misdemeanor_count',
       'juvenile_other_offense_count', 'prior_offense_count',
       'days_between_screening_and_arrest',
       'days_from_compas_to_current_offense', 'is_recidivist', 'arrest_to_rec',
       'is_violent_recidivist', 'charge_id', 'rec_charge_id', 'inmate_id',
       'sentence_type_id', 'state_id', 'recidivism_offense_date_id',
       'current_jail_entry_date_id', 'current_jail_release_date_id',
       'screening_date_id', 'recidivism_risk_score_id',
       'violence_risk_score_id'],
      dtype='object')
Index(['Unnamed: 0', ' Case_id', 'name', 'sex', 'date_of_birth', 'age', 'race',
       'juvenile_felony_count', 'risk_decile_score',
       'juvenile_misdemeanor_count', 'juvenile_other_offense_count',
       'prior_offense_count', 'days_between_screening_and_arrest',
       'current_jail_entry_date', 'current_jail_release_date',
       'days_from_compas_to_current_offense', 'current_charge_degree',
       'curr

In [83]:
# Strip leading/trailing spaces from column names
cases.columns = cases.columns.str.strip()
old_cases.columns = old_cases.columns.str.strip()

# Merge cases with old_cases based on 'Case_id'
merged_cases = cases.merge(old_cases[['Case_id', 'screening_date', 'current_jail_entry_date', 'current_jail_release_date', 'recidivism_offense_date']],
                           on='Case_id', how='left')


In [85]:
print(merged_cases.head())


   Case_id  juvenile_felony_count  juvenile_misdemeanor_count  \
0        0                      0                           0   
1        1                      0                           0   
2        2                      0                           0   
3        3                      0                           0   
4        4                      0                           1   

   juvenile_other_offense_count  prior_offense_count  \
0                             0                    0   
1                             0                    0   
2                             0                    0   
3                             1                    4   
4                             0                    1   

   days_between_screening_and_arrest  days_from_compas_to_current_offense  \
0                                 -1                                    1   
1                                 -1                                   -1   
2                                 -1     

In [87]:
# Strip column names for both DataFrames
cases.columns = cases.columns.str.strip()
old_cases.columns = old_cases.columns.str.strip()

# Merge the cases DataFrame with old_cases based on 'Case_id'
merged_cases = cases.merge(old_cases[['Case_id', 'screening_date', 'current_jail_entry_date', 
                                      'current_jail_release_date', 'recidivism_offense_date']],
                           on='Case_id', how='left')

# Drop the date ID columns, keeping only the actual dates
merged_cases = merged_cases.drop(['screening_date_id', 'current_jail_entry_date_id', 
                                  'current_jail_release_date_id', 'recidivism_offense_date_id'], axis=1)

# Check the DataFrame to ensure changes
print(merged_cases.head())


   Case_id  juvenile_felony_count  juvenile_misdemeanor_count  \
0        0                      0                           0   
1        1                      0                           0   
2        2                      0                           0   
3        3                      0                           0   
4        4                      0                           1   

   juvenile_other_offense_count  prior_offense_count  \
0                             0                    0   
1                             0                    0   
2                             0                    0   
3                             1                    4   
4                             0                    1   

   days_between_screening_and_arrest  days_from_compas_to_current_offense  \
0                                 -1                                    1   
1                                 -1                                   -1   
2                                 -1     

In [91]:
merged_cases.to_csv('dwh_csv/f_cases.csv', index = False)

In [95]:
# Strip column names for both DataFrames (if needed)
inmates.columns = inmates.columns.str.strip()
dates.columns = dates.columns.str.strip()

# Merge inmates with dates based on 'dob_id' and 'date_id'
inmates_merged = inmates.merge(dates[['date_id', 'day', 'month', 'year']], 
                               left_on='dob_id', right_on='date_id', how='left')

# Create a 'dob' (Date of Birth) column by concatenating 'year', 'month', 'day'
inmates_merged['dob'] = pd.to_datetime(inmates_merged[['year', 'month', 'day']])

# Drop unnecessary columns: 'dob_id', 'date_id', 'day', 'month', 'year'
inmates_merged = inmates_merged.drop(['dob_id', 'date_id', 'day', 'month', 'year'], axis=1)

# Check the updated inmates DataFrame
print(inmates_merged.head())


   inmate_id          fullname  age  gender_id  race_id  age_group_id  \
0          1  miguel hernandez   77          1        1           5.0   
1          2      michael ryan   39          1        2           2.0   
2          3       kevon dixon   42          1        3           2.0   
3          4          ed philo   33          1        3           1.0   
4          5       marcu brown   31          1        3           1.0   

         dob  
0 1947-04-18  
1 1985-02-06  
2 1982-01-22  
3 1991-05-14  
4 1993-01-21  


In [97]:
inmates_merged.to_csv('dwh_csv/d_inmate.csv', index = False)

In [102]:
# Strip column names for both DataFrames (if needed)
general.columns = general.columns.str.strip()
dates.columns = dates.columns.str.strip()

# Merge general with dates based on 'date_id'
general_merged = general.merge(dates[['date_id', 'day', 'month', 'year']], 
                               left_on='date_id', right_on='date_id', how='left')

# Create a 'date' column by concatenating 'year', 'month', 'day'
general_merged['date'] = pd.to_datetime(general_merged[['year', 'month', 'day']])

# Drop unnecessary columns: 'date_id', 'day', 'month', 'year'
general_merged = general_merged.drop(['date_id', 'day', 'month', 'year'], axis=1)

# Check the updated general DataFrame
print(general_merged.head())


   population  unemployment_rate  participation_rate  poverty_nb  \
0         NaN                6.7                57.0         NaN   
1         NaN                7.2                58.2         NaN   
2         NaN                6.4                59.0         NaN   
3         NaN                7.2                59.5         NaN   
4         NaN                8.8                59.4         NaN   

   poverty_percentage  prisoners_count  violent_crimes_total  \
0                 NaN              NaN                   NaN   
1                 NaN              NaN                   NaN   
2                 NaN              NaN                   NaN   
3                 NaN              NaN                   NaN   
4                 NaN              NaN                   NaN   

   murder_manslaughter  robbery  agg_assault  property_crime_total  burglary  \
0                  NaN      NaN          NaN                   NaN       NaN   
1                  NaN      NaN          NaN  

In [106]:
general_merged.to_csv('dwh_csv/f_general_metrics.csv', index=False)